# Fiddler Simple Monitoring Quick Start Guide

Fiddler is the pioneer in enterprise Model Performance Management (MPM), offering a unified platform that enables Data Science, MLOps, Risk, Compliance, Analytics, and other LOB teams to **monitor, analyze, and improve ML deployments at enterprise scale**.
Obtain contextual insights at any stage of the ML lifecycle, improve predictions, increase transparency and fairness, and optimize business revenue.

---

You can start using Fiddler ***in minutes*** by following these quick steps:

1. Connect to Fiddler
2. Load a Data Sample
3. Define the Model Specifications
4. Set the Model Task
5. Create a Model
6. Set Up Alerts **(Optional)**
7. Create a Custom Metric **(Optional)**
8. Create a Segment **(Optional)**
9. Publish a Pre-production Baseline **(Optional)**
10. Configure a Rolling Baseline **(Optional)**
11. Publish Production Events

# 0. Imports

In [2]:
%pip install -q fiddler-client

import time as time

import pandas as pd
import fiddler as fdl

print(f'Running Fiddler Python client version {fdl.__version__}')


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Running Fiddler Python client version 3.4.0


## 1. Connect to Fiddler

Before you can add information about your model with Fiddler, you'll need to connect using the Fiddler Python client.


---


**We need a couple pieces of information to get started.**
1. The URL you're using to connect to Fiddler
2. Your authorization token

Your authorization token can be found by navigating to the **Credentials** tab on the **Settings** page of your Fiddler environment.

In [3]:
URL = ''  # Make sure to include the full URL (including https:// e.g. 'https://your_company_name.fiddler.ai').
TOKEN = ''

Constants for this example notebook, change as needed to create your own versions

In [7]:
PROJECT_NAME = 'quickstart_examples'  # If the project already exists, the notebook will create the model under the existing project.
MODEL_NAME = 'bank_churn_simple_monitoring'

STATIC_BASELINE_NAME = 'baseline_dataset'
ROLLING_BASELINE_NAME = 'rolling_baseline_1week'

# Sample data hosted on GitHub
PATH_TO_SAMPLE_CSV = 'https://raw.githubusercontent.com/fiddler-labs/fiddler-examples/main/quickstart/data/v3/churn_data_sample.csv'
PATH_TO_EVENTS_CSV = 'https://raw.githubusercontent.com/fiddler-labs/fiddler-examples/main/quickstart/data/v3/churn_production_data.csv'

Now just run the following to connect to your Fiddler environment.

In [5]:
fdl.init(url=URL, token=TOKEN)

#### 1.a Create New or Load Existing Project

Once you connect, you can create a new project by specifying a unique project name in the fld.Project constructor and calling the `create()` method. If the project already exists, it will load it for use.

In [8]:
try:
    # Create project
    project = fdl.Project(name=PROJECT_NAME).create()
    print(f'New project created with id = {project.id} and name = {project.name}')
except fdl.Conflict:
    # Get project by name
    project = fdl.Project.from_name(name=PROJECT_NAME)
    print(f'Loaded existing project with id = {project.id} and name = {project.name}')

New project created with id = 70b74177-c712-44b1-b431-2377c1b908ab and name = quickstart_examplesx


You should now be able to see the newly created project in the Fiddler UI.

<table>
    <tr>
        <td>
            <img src="https://raw.githubusercontent.com/fiddler-labs/fiddler-examples/main/quickstart/images/simple_monitoring_1.png" />
        </td>
    </tr>
</table>

## 2. Load a Data Sample

In this example, we'll be considering the case where we're a bank and we have **a model that predicts churn for our customers**.
  
In order to get insights into the model's performance, **Fiddler needs a small sample of data** to learn the schema of incoming data.

In [9]:
sample_data_df = pd.read_csv(PATH_TO_SAMPLE_CSV)
column_list  = sample_data_df.columns
sample_data_df

,customer_id,creditscore,geography,gender,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,predicted_churn,churn,timestamp
0,27acd1c2,545,Texas,Male,37,9,110483.86,1,1,1,127394.67,0.897202,yes,1710428231855
1,27b36d0c,497,Texas,Female,55,7,131778.66,1,1,1,9972.64,0.997441,yes,1710428262096
2,27b5360a,509,New York,Female,29,0,107712.57,2,1,1,92898.17,0.920563,yes,1710428292338
3,27b5d650,743,Hawaii,Nonbinary,39,6,0.00,2,1,0,44265.28,0.779282,yes,1710428322579
4,27b236a8,699,Florida,Female,25,8,0.00,2,1,1,52404.47,0.825474,yes,1710428352821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,27b409ba,686,Texas,Male,39,3,129626.19,2,1,1,103220.56,0.760645,yes,1711032910888
19996,27aaff96,446,Massachusetts,Female,45,10,125191.69,1,1,1,128260.86,0.216093,no,1711032941130
19997,27ad3162,794,California,Male,35,6,0.00,2,1,1,68730.91,0.982021,yes,1711032971371
19998,27b076ce,832,California,Male,61,2,0.00,1,0,1,127804.66,0.071598,no,1711033001613


## 3. Define the Model Specifications

In order to create a model in Fiddler, create a ModelSpec object with information about what each column of your data sample should used for.

Fiddler supports four column types:
1. **Inputs**
2. **Outputs** (Model predictions)
3. **Target** (Ground truth values)
4. **Metadata**

In [10]:
input_columns = list(
    column_list.drop(['predicted_churn', 'churn', 'customer_id', 'timestamp'])
)

In [11]:
model_spec = fdl.ModelSpec(
    inputs=input_columns,
    outputs=['predicted_churn'],
    targets=['churn'],  # Note: only a single Target column is allowed, use metadata columns and custom metrics for additional targets
    metadata=['customer_id', 'timestamp'],
)

If you have columns in your ModelSpec which denote **prediction IDs or timestamps**, then Fiddler can use these to power its analytics accordingly.

Let's call them out here and use them when configuring the Model in step 5.

In [12]:
id_column = 'customer_id'
timestamp_column = 'timestamp'

## 4. Set the Model Task

Fiddler supports a variety of model tasks. In this case, we're adding a binary classification model.

For this, we'll create a ModelTask object and an additional ModelTaskParams object to specify the ordering of our positive and negative labels.

*For a detailed breakdown of all supported model tasks, click [here](https://docs.fiddler.ai/product-guide/task-types).*

In [13]:
model_task = fdl.ModelTask.BINARY_CLASSIFICATION

task_params = fdl.ModelTaskParams(target_class_order=['no', 'yes'])

## 5. Create a Model

Create a Model object and publish it to Fiddler, passing in
1. Your data sample
2. The ModelSpec object
3. The ModelTask and ModelTaskParams objects
4. The ID and timestamp columns

In [14]:
model = fdl.Model.from_data(
    name=MODEL_NAME,
    project_id=project.id,
    source=sample_data_df,
    spec=model_spec,
    task=model_task,
    task_params=task_params,
    event_id_col=id_column,
    event_ts_col=timestamp_column,
)

model.create()
print(f'New model created with id = {model.id} and name = {model.name}')

New model created with id = 9e1a73c7-c0ee-4fd9-961c-a404368212fb and name = bank_churn_simple_monitoring


On the project page, you should now be able to see the newly onboarded model with its model schema.

<table>
    <tr>
        <td>
            <img src="https://github.com/fiddler-labs/fiddler-examples/blob/main/quickstart/images/simple_monitoring_3.png?raw=true" />
        </td>
    </tr>
</table>

<table>
    <tr>
        <td>
            <img src="https://github.com/fiddler-labs/fiddler-examples/blob/main/quickstart/images/simple_monitoring_4.png?raw=true" />
        </td>
    </tr>
</table>

## 6. Set Up Alerts (Optional)

Fiddler allows creating alerting rules when your data or model predictions deviate from expected behavior.

The alert rules can compare metrics to **absolute** or **relative** values.

Please refer to [our documentation](https://docs.fiddler.ai/client-guide/alerts-with-fiddler-client) for more information on Alert Rules.

---
  
Let's set up some alert rules.

The following API call sets up a Data Integrity type rule which triggers an email notification when published events have 2 or more range violations in any 1 day bin for the `numofproducts` column.

In [15]:
alert_rule_1 = fdl.AlertRule(
    name='Bank Churn Range Violation Alert dupe',
    model_id=model.id,
    metric_id='range_violation_count',
    bin_size=fdl.BinSize.DAY,
    compare_to=fdl.CompareTo.RAW_VALUE,
    priority=fdl.Priority.HIGH,
    warning_threshold=2,
    critical_threshold=3,
    condition=fdl.AlertCondition.GREATER,
    columns=['numofproducts'],
)

alert_rule_1.create()
print(
    f'New alert rule created with id = {alert_rule_1.id} and name = {alert_rule_1.name}'
)

# Set notification configuration for the alert rule, a single email address for this simple example
alert_rule_1.set_notification_config(emails=['name@google.com'])

New alert rule created with id = 1d75a6f9-ecea-4d57-be40-030af4b2a257 and name = Bank Churn Range Violation Alert dupe


NotificationConfig(emails=['name@google.com'], pagerduty_services=[], pagerduty_severity='', webhooks=[])

Let's add a second alert rule.

This one sets up a Performance type rule which triggers an email notification when precision metric is 5% higher than that from 1 hr bin one day ago.

In [16]:
alert_rule_2 = fdl.AlertRule(
    name='Bank Churn Performance Alert dupe',
    model_id=model.id,
    metric_id='precision',
    bin_size=fdl.BinSize.HOUR,
    compare_to=fdl.CompareTo.TIME_PERIOD,
    compare_bin_delta=24,  # Multiple of the bin size
    condition=fdl.AlertCondition.GREATER,
    warning_threshold=0.05,
    critical_threshold=0.1,
    priority=fdl.Priority.HIGH,
)

alert_rule_2.create()
print(
    f'New alert rule created with id = {alert_rule_2.id} and name = {alert_rule_2.name}'
)

# Set notification configuration for the alert rule, a single email address for this simple example
alert_rule_2.set_notification_config(emails=['name@google.com'])

New alert rule created with id = 5a6c9c05-9ba5-4c47-b065-ca0e8b6b5ca7 and name = Bank Churn Performance Alert dupe


NotificationConfig(emails=['name@google.com'], pagerduty_services=[], pagerduty_severity='', webhooks=[])

## 7. Create a Custom Metric (Optional)

Fiddler's [Custom Metric API](https://docs.fiddler.ai/python-client-3-x/api-methods-30#custommetric) enables user-defined formulas for custom metrics.  Custom metrics will be tracked over time and can be used in Charts and Alerts just like the many out of the box metrics provided by Fiddler.  Custom metrics can also be managed in the Fiddler UI.

Please refer to [our documentation](https://docs.fiddler.ai/product-guide/monitoring-platform/custom-metrics) for more information on Custom Metrics.

---
  
Let's create an example custom metric.

In [17]:
custom_metric = fdl.CustomMetric(
    name='Lost Revenue dupe',
    model_id=model.id,
    description='A metric to track revenue lost for each false positive prediction.',
    definition="""sum(if(fp(),1,0) * -100)""",  # This is an excel like formula which adds -$100 for each false positive predicted by the model
)

custom_metric.create()
print(
    f'New custom metric created with id = {custom_metric.id} and name = {custom_metric.name}'
)

New custom metric created with id = 68fae885-db51-43cb-adee-ea4b6722ec01 and name = Lost Revenue dupe


## 8. Create a Segment (Optional)
Fiddler's [Segment API](https://docs.fiddler.ai/python-client-3-x/api-methods-30#segments) enables defining named cohorts/sub-segments in your production data. These segments can be tracked over time, added to charts, and alerted upon. Segments can also be managed in the Fiddler UI.

Please refer to our [documentation](https://docs.fiddler.ai/product-guide/monitoring-platform/segments) for more information on the creation and management of segments.

Let's create a segment to track customers from Hawaii for a specific age range.

In [18]:
segment = fdl.Segment(
    name='Hawaii Customers between 30 and 60 dupe',
    model_id=model.id,
    description='Hawaii Customers between 30 and 60',
    definition="(age<60 or age>30) and geography=='Hawaii'",
)

segment.create()
print(f'New segment created with id = {segment.id} and name = {segment.name}')

New segment created with id = d17e20a1-2cd0-4935-935d-59be6dddeb2c and name = Hawaii Customers between 30 and 60 dupe


## 9. Publish a Static Baseline (Optional)

Since Fiddler already knows how to process data for your model, we can now add a **baseline dataset**.

You can think of this as a static dataset which represents **"golden data,"** or the kind of data your model expects to receive.

Then, once we start sending production data to Fiddler, you'll be able to see **drift scores** telling you whenever it starts to diverge from this static baseline.

***

Let's publish our **original data sample** as a pre-production dataset. This will automatically add it as a baseline for the model.


*For more information on how to design your baseline dataset, [click here](https://docs.fiddler.ai/client-guide/creating-a-baseline-dataset).*

In [19]:
baseline_publish_job = model.publish(
    source=sample_data_df,
    environment=fdl.EnvType.PRE_PRODUCTION,
    dataset_name=STATIC_BASELINE_NAME,
)
print(
    f'Initiated pre-production environment data upload with Job ID = {baseline_publish_job.id}'
)

# Uncomment the line below to wait for the job to finish, otherwise it will run in the background.
# You can check the status on the Jobs page in the Fiddler UI or use the job ID to query the job status via the API.
# baseline_publish_job.wait()

Initiated pre-production environment data upload with Job ID = d055bef9-c422-4f8e-908c-65e83b298cdb


## 10. Configure a Rolling Baseline (Optional)

Fiddler also allows you to configure a baseline based on **past production data.**

This means instead of looking at a static slice of data, it will look into past production events and use what it finds for drift calculation.

Please refer to [our documentation](https://docs.fiddler.ai/client-guide/creating-a-baseline-dataset) for more information on Baselines.

---
  
Let's set up a rolling baseline that will allow us to calculate drift relative to production data from 1 week back.

In [20]:
rolling_baseline = fdl.Baseline(
    model_id=model.id,
    name=ROLLING_BASELINE_NAME,
    type_=fdl.BaselineType.ROLLING,
    environment=fdl.EnvType.PRODUCTION,
    window_bin_size=fdl.WindowBinSize.DAY,  # Size of the sliding window
    offset_delta=7,  # How far back to set our window (multiple of window_bin_size)
)

rolling_baseline.create()
print(
    f'New rolling baseline created with id = {rolling_baseline.id} and name = {rolling_baseline.name}'
)

New rolling baseline created with id = 24f9e275-d455-4871-9034-084d84514abd and name = rolling_baseline_1week


## 11. Publish Production Events

Finally, let's send in some production data!


Fiddler will **monitor this data and compare it to your baseline to generate powerful insights into how your model is behaving**.


---


Each record sent to Fiddler is called **an event**.
  
Let's load some sample events from a CSV file.

In [21]:
production_data_df = pd.read_csv(PATH_TO_EVENTS_CSV)

# Shift the timestamps of the production events to be as recent as today
production_data_df['timestamp'] = production_data_df['timestamp'] + (
    int(time.time() * 1000) - production_data_df['timestamp'].max()
)
production_data_df

,customer_id,creditscore,geography,gender,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,predicted_churn,churn,timestamp
0,27c349a2,559,California,Male,52,2,0.00,1,1,0,129013.59,0.007448,no,1731082075444
1,27c35cee,482,California,Male,55,5,97318.25,1,0,1,78416.14,0.804852,yes,1731084504359
2,27c364f0,651,Florida,Female,46,4,89743.05,1,1,0,156425.57,0.012754,no,1731086933275
3,27c3627a,611,Hawaii,Male,38,7,0.00,1,1,1,63202.00,0.882252,yes,1731089362190
4,27c34164,696,California,Female,33,4,0.00,2,1,1,73371.65,0.999736,yes,1731091791106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,27c350b4,781,Hawaii,Female,48,0,57098.96,6,1,0,85644.06,0.032330,no,1731677159781
246,27c357e4,797,Hawaii,Female,55,10,0.00,9,1,1,49418.87,0.020316,no,1731679588697
247,27c36216,554,Hawaii,Male,31,1,0.00,7,0,1,192660.55,0.269628,yes,1731682017612
248,27c34d12,701,Hawaii,Nonbinary,37,1,0.00,7,1,0,163457.55,0.769625,yes,1731684446528


In [22]:
production_publish_job = model.publish(production_data_df)

print(
    f'Initiated production environment data upload with Job ID = {production_publish_job.id}'
)

# Uncomment the line below to wait for the job to finish, otherwise it will run in the background.
# You can check the status on the Jobs page in the Fiddler UI or use the job ID to query the job status via the API.
# production_publish_job.wait()

Initiated production environment data upload with Job ID = 4b2b8c7b-023b-4ffe-b8c0-1423b4ed736a


# Get Insights
  
Return to your Fiddler environment to get enhanced observability into your model's performance.

<table>
    <tr>
        <td>
            <img src="https://raw.githubusercontent.com/fiddler-labs/fiddler-examples/main/quickstart/images/simple_monitoring_5.png" />
        </td>
    </tr>
</table>

**What's Next?**

Try the [LLM Monitoring - Quick Start Notebook](https://docs.fiddler.ai/quickstart-notebooks/simple-llm-monitoring)

---


**Questions?**  
  
Check out [our docs](https://docs.fiddler.ai/) for a more detailed explanation of what Fiddler has to offer.

Join our [community Slack](http://fiddler-community.slack.com/) to ask any questions!

If you're still looking for answers, fill out a ticket on [our support page](https://fiddlerlabs.zendesk.com/) and we'll get back to you shortly.